# Tutorial 4 - Pioneer
This tutorial models a potential design for a WEC used to provide power to instruments in the [Pioneer Central Surface Mooring System](https://oceanobservatories.org/site/cp01cnsm/) within the [Coastal Pioneer Array](https://oceanobservatories.org/array/coastal-pioneer-array/) off the coast of New England. This system has instrumentation to measure various meteorological, surface, near-surface, and seabed phenomena on the Continental Shelf-Slope and transmit back to shore.

Unlike previous tutorials, this tutorial does not include an outer optimization loop. Instead, the focus here is on effectively modeling an optimization problem for the control states of a unique WEC archetype with several co-dependent components. Topics include:

* Expanding the control state vector
* Custom PTO physics
* Modeling non-hydrodynamic degrees of freedom
<p><img src=https://oceanobservatories.org/wp-content/uploads/2015/09/Central-Surface-Mooring-Array-01-1.png alt="Diagram of the Pioneer device" width="307">

In [ ]:
import os
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

import wecopttool as wot
wot.set_loglevel('WARNING')

## Given properties

In [ ]:
buoy_props = {
    'CG': 0.242, # m
    'MOI': 7484., # kg-m^2
    'f0': 2 * np.pi /3.0842, # rad/s
}

flywheel_props = {
    'MOI': 25,  # kg-m^2
    'Coulomb friction': 2.0,  # N-m
    'Viscous friction': 0.02,  # N-m/rad
    'Motor damping': 4.0, # N-m-s/rad
}

spring_props = {
    'Max torque': 750, # N-m
    'Max displacement': np.deg2rad(45.0), # rad
}

## Geometry
Same general shape as WaveBot

In [ ]:
in2m = 0.0254 # inch -> meter conversion factor
draft = 0.5 # meters
freeboard = 40. * in2m - draft
hull_geom = wot.geom.WaveBot(r1=130./2 * in2m,
                             r2=110./2 * in2m, 
                             h1=22.679 * in2m,
                             h2=17.321 * in2m,
                             scale_factor=1,
                             freeboard=freeboard)
mesh = hull_geom.mesh(mesh_size_factor=0.5)
_ = hull_geom.plot_cross_section()

### Floating body
1DOF in pitch. The `FloatingBody` object in Capytaine only models the buoy since it is the only body being excited by the waves; the flywheel dynamics will be captured later on.

In [ ]:
pnr_fb = cpy.FloatingBody.from_meshio(mesh, name="Pioneer")
pnr_fb.add_rotation_dof(name='Pitch')
pnr_fb.center_of_mass = np.array([0., 0., buoy_props['CG']])
pnr_fb.rotation_center = pnr_fb.center_of_mass
ndof = pnr_fb.nb_dofs
pnr_fb.show_matplotlib()

## Hydrodynamics and hydrostatics
Hydrostatic stiffness values match closely to experimental values

In [ ]:
rho = 1025. # kg/m^3
f1 = 0.025 # Hz
nfreq = 25
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency


file_name = "data/pioneer_bem_25freq.nc"
if os.path.exists(file_name):
    bem_data = wot.read_netcdf(file_name)
else:    
    bem_data = wot.run_bem(pnr_fb, freq)
    wot.write_netcdf(file_name, bem_data)

pnr_fb.keep_immersed_part()
k_buoy = pnr_fb.compute_hydrostatic_stiffness(rho=rho).values.squeeze()
k_spring = spring_props['Max torque'] / spring_props['Max displacement']
print(f'Hydrostatic stiffness from Capytaine: {k_buoy} N-m/rad')
print('Hydrostatic stiffness from experiment: 37204 N-m/rad')

omega = bem_data.omega.values
omega_r = np.sqrt(k_buoy / buoy_props['MOI'])

In [ ]:
# plots
fig_am, ax_am = plt.subplots(tight_layout=True, sharex=True)
fig_rd, ax_rd = plt.subplots(tight_layout=True, sharex=True)
fig_ex, ax_ex = plt.subplots(tight_layout=True, sharex=True)

# Excitation
np.abs(bem_data.diffraction_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashed', label='Diffraction force')
np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashdot', label='Froude-Krylov force')
ex_handles, ex_labels = ax_ex.get_legend_handles_labels()
ax_ex.set_xlabel(f'$\omega$', fontsize=10)
ax_ex.set_title('Wave Excitation Coefficients', fontweight='bold')
fig_ex.legend(ex_handles, ex_labels, loc='center right', frameon=False)

# Added mass
bem_data.added_mass.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_am)
ax_am.set_xlabel(f'$\omega$', fontsize=10)
ax_am.set_title('Added Mass Coefficients', fontweight='bold')

# Radiation damping
bem_data.radiation_damping.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_rd)
ax_rd.set_xlabel(f'$\omega$', fontsize=10)
ax_rd.set_title('Radiation Damping Coefficients', fontweight='bold')

In [ ]:
# bem_data.added_mass[:] = 1480.
# bem_data.radiation_damping[:] = 2289.

## PTO
Using same impedance matrix as WaveBot, except replace gear ratio with the one determined from the spring stiffness and buoy moment of inertia + natural frequency.

In [ ]:
nstate_pto = 2 * nfreq

def gear_ratio(k_spring, moi, nat_freq):
    return np.sqrt(k_spring / (moi * nat_freq**2))
# n_default = gear_ratio(k_spring, flywheel_props['MOI'], buoy_props['f0'])
n_default = 3.

In [ ]:
torque_constant = 3.512
winding_resistance = 0.304
winding_inductance = 0.0
drivetrain_inertia = 0.0
drivetrain_friction = 0.0
drivetrain_stiffness = 0.0

drivetrain_impedance = (1j*omega*drivetrain_inertia + 
                        drivetrain_friction + 
                        1/(1j*omega)*drivetrain_stiffness) 

winding_impedance = winding_resistance + 1j*omega*winding_inductance


z_11 = np.array([[-1* n_default**2 * drivetrain_impedance]])
off_diag = np.sqrt(3.0/2.0) * torque_constant * n_default
z_12 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_21 = np.array([[-1*(off_diag+0j) * np.ones(omega.shape)]])
z_22 = np.array([[winding_impedance]])
z_12_inv = np.linalg.inv(z_12.T).T

mmult = lambda a,b: np.einsum('mnr,mnr->mnr', a, b)
abcd_11 = -1 * mmult(z_12_inv, z_11)
abcd_12 = z_12_inv
abcd_21 = z_21 - mmult(z_22, mmult(z_12_inv, z_11))
abcd_22 = mmult(z_22, z_12_inv)

row_1 = np.hstack([abcd_11, abcd_12])
row_2 = np.hstack([abcd_21, abcd_22])
abcd = np.vstack([row_1, row_2])

In [ ]:
def block(re, im): return np.array([[re, -im], [im, re]])
for idof in range(2):
    for jdof in range(2):
        Zp = abcd[idof, jdof, :]
        re = np.real(Zp)
        im = np.imag(Zp)
        # Exclude the sine component of the 2-point wave
        blocks = [block(ire, iim) for (ire, iim) in zip(re[:-1], im[:-1])]
        # re[0] added for the zero frequency power loss (DC), could be re[n]
        blocks = [re[0]] + blocks + [re[-1]]
        if jdof==0:
            row = block_diag(*blocks)
        else:
            row = np.hstack([row, block_diag(*blocks)])
    if idof==0:
        transfer_mat = row
    else:
        transfer_mat = np.vstack([transfer_mat, row])
# transfer_mat = None

In [ ]:
def x_rel(wec, x_wec, x_opt):
    pos_wec = wec.vec_to_dofmat(x_wec)
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    return pos_wec - pos_fw

def rel_position(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, pos_rel)

def rel_velocity(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_rel = x_rel(wec, x_wec, x_opt)
    vel_rel = np.dot(wec.derivative_mat, pos_rel)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, vel_rel)

def f_motor(wec, x_wec, x_opt, waves, nsubsteps=1):
    # vel_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    # return -4 * vel_td
    motor = np.reshape(x_opt[:nstate_pto], (-1, ndof), order='F')
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, motor)

def mechanical_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    force_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    vel_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return vel_td * force_td

def mechanical_energy(wec, x_wec, x_opt, waves, nsubsteps=1):
    power_td = mechanical_power(wec, x_wec, x_opt, waves, nsubsteps)
    return np.sum(power_td) * wec.dt/nsubsteps

def mechanical_average_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    energy = mechanical_energy(wec, x_wec, x_opt, waves, nsubsteps)
    return energy / wec.tf

def power(wec, x_wec, x_opt, waves, nsubsteps=1):
    if transfer_mat is not None:
        q1_td = rel_velocity(wec, x_wec, x_opt, waves)
        e1_td = f_motor(wec, x_wec, x_opt, waves)
        q1 = wot.complex_to_real(wec.td_to_fd(q1_td, False))
        e1 = wot.complex_to_real(wec.td_to_fd(e1_td, False))
        vars_1 = np.hstack([q1, e1])
        vars_1_flat = wec.dofmat_to_vec(vars_1)
        vars_2_flat = np.dot(transfer_mat, vars_1_flat)
        vars_2 = wot.vec_to_dofmat(vars_2_flat, 2)
        q2 = vars_2[:, 0]
        e2 = vars_2[:, 1]
        time_mat = wec.time_mat_nsubsteps(nsubsteps)
        q2_td = np.dot(time_mat, q2)
        e2_td = np.dot(time_mat, e2)
    else:
        q2_td = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
        e2_td = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return q2_td * e2_td

def energy(wec, x_wec, x_opt, waves, nsubsteps=1):
    power_td = power(wec, x_wec, x_opt, waves, nsubsteps)
    return np.sum(power_td) * wec.dt/nsubsteps

def average_power(wec, x_wec, x_opt, waves, nsubsteps=1):
    e = energy(wec, x_wec, x_opt, waves, nsubsteps)
    return e / wec.tf

In previous tutorials, all the relevant WEC dynamics have been captured in the `x_wec` vector. However, `x_wec` assumes all the degrees of freedom are excited by waves, which is not the case with the flywheel in the Pioneer model. Since we need to capture the flywheel dynamics to properly model the PTO, we work around this by including the position states of the flywheel as appended entries in `x_opt`, and overwrite the motion functions within the `PTO` module to capture the relative motion of the flywheel so the power calculations within that module are correct. This is a dirty hack for now -- this will be more cleanly implemented in an upcoming release, and this tutorial will be updated accordingly.

### Constraints
 * **Peak torque** - Peak torque
 * **Peak RMS speed** - Peak RMS speed

In [ ]:
torque_peak_max = 50.
permitted_rms_speed = 3300 * 2*np.pi / 60 # rad/s
def const_peak_torque_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    torque = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

def const_rms_velocity_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    pos = wec.vec_to_dofmat(x_opt[nstate_pto:])
    pto_vel = np.dot(wec.derivative_mat, pos)
    rms_vel = np.sqrt(np.mean(pto_vel**2))
    return permitted_rms_speed - rms_vel

### Additional forces
 * **Flywheel friction** - Flywheel friction
 * **Magnetic spring** - Magnetic spring
 * **Flywheel inertia** - Not really a force, but captured here for the flywheel residual function below

In [ ]:
def f_spring(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps)
    return -k_spring * (1/n_default)**2 * rel_pos

def f_friction(wec, x_wec, x_opt, waves, nsubsteps=1):
    rel_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return (
        np.tanh(rel_vel) * flywheel_props['Coulomb friction']
      + rel_vel * flywheel_props['Viscous friction']
    )

def fw_inertia(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
    acc_fw = np.dot(wec.derivative2_mat, pos_fw)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    acc_fw = np.dot(time_matrix, acc_fw)
    return flywheel_props['MOI'] * acc_fw

#### Residual
Equality constraint of Newton's 2nd law on the flywheel (i.e. $r(x) = I \alpha - \tau = 0 $). Same basic structure as residual for the WEC dynamics as described in the [WecOptTool theory documentation](https://sandialabs.github.io/WecOptTool/theory.html).

In [ ]:
def const_flywheel_resid(wec, x_wec, x_opt, waves):
    return (fw_inertia(wec, x_wec, x_opt, waves)
          + f_spring(wec, x_wec, x_opt, waves)
          - f_friction(wec, x_wec, x_opt, waves)
          + f_motor(wec, x_wec, x_opt, waves)
          ).flatten()
    

### WEC object
Finally, we can use all the different components we've developed thus far to construct a `WEC` object:

In [ ]:
f_add = {
    'Spring': f_spring,
    'Motor': f_motor,
}

constraints = [
    {'type': 'ineq', 'fun': const_peak_torque_pto},
    {'type': 'ineq', 'fun': const_rms_velocity_pto},
    {'type': 'eq', 'fun': const_flywheel_resid},
]

In [ ]:
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=np.array([[buoy_props['MOI']]]),
                       hydrostatic_stiffness=k_buoy,
                       f_add=f_add,
                       constraints=constraints,
                       dof_names=bem_data.influenced_dof.values,
                        )

## Waves
A regular wave will allow us to get a good initial understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = 0.15
# wavefreq =  1/5.3
wavefreq =  omega_r / (2*np.pi)
phase = 0
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)
# spectrum = lambda f: wot.waves.jonswap_spectrum(freq=f, fp=wavefreq, hs=2*amplitude)
# efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
# waves = wot.waves.long_crested_wave(efth)
# efth.plot(marker='.')

## Solve
Note the size of the `nstate_opt` variable compared to previous tutorials. The first half of the variable includes the PTO force states at each Fourier component, as in previous tutorials (the `nstate_pto` component). The second half is the flywheel positions at each Fourier component, which is basically an analog to `wec.nstate_wec`.

In [ ]:
obj_fun = average_power
nstate_opt = nstate_pto + wec.nstate_wec
options = {'maxiter': 500}
results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    x_wec_0=np.ones(wec.nstate_wec)*0.8,
    x_opt_0=np.ones(nstate_opt),
    scale_x_wec=1e1,
    scale_x_opt=1e-2,
    scale_obj=1e-2,
    optim_options=options
)

## Post-process and plotting

In [ ]:
results.fun

In [ ]:
nsubsteps = 5
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=nsubsteps)
force_excitation = (wec_tdom.force.sel(type='Froude_Krylov') + 
                    wec_tdom.force.sel(type='diffraction'))
exc_torque = force_excitation.squeeze().values

In [ ]:
x_wec, x_opt = wot.decompose_state(results.x, 1, nfreq)
pto_pos = rel_position(wec, x_wec, x_opt, waves, nsubsteps) / n_default**2
pto_vel = rel_velocity(wec, x_wec, x_opt, waves, nsubsteps)
pto_force = f_motor(wec, x_wec, x_opt, waves, nsubsteps)
pto_force_fd = wec.td_to_fd(pto_force[::nsubsteps])
pto_mech_power = mechanical_power(wec, x_wec, x_opt, waves, nsubsteps)
fw_pos = np.dot(wec.time_mat_nsubsteps(nsubsteps), x_opt[nstate_pto:])

In [ ]:
# comparison is 3x as long as WecOptTool simulation, and the 2/3rds of
# the results will be removed later to get rid of the startup transient
dt = wec_tdom.time.values[1] - wec_tdom.time.values[0]
comp_t = np.concatenate([wec_tdom.time.values,
                         wec_tdom.time.values[-1] + wec_tdom.time.values + dt])
comp_t = np.concatenate([comp_t,
                         comp_t[-1] + wec_tdom.time.values + dt])
comp_exc_torque = np.concatenate([exc_torque, exc_torque, exc_torque])
len_t = len(wec_tdom.time.values)
k1 = k_buoy
k2 = k_spring
N1 = 1 / n_default  # inverse of gear ratio
# N2 = 1; # no gear on PTO damper
J1 = buoy_props['MOI'] + bem_data.added_mass.squeeze().values[11]
J2 = flywheel_props['MOI']
B1 = bem_data.radiation_damping.squeeze().values[11]
B2 = flywheel_props['Motor damping']

def subPolyFrac(p1num, p1den, p2num, p2den):
    num = np.polysub(np.polymul(p1num, p2den), np.polymul(p2num, p1den))
    den = np.polymul(p1den, p2den)

    return num, den

# Buoy angle to flywheel transfer function (theta_2 / theta1)
# Equation 3 Jantzen "Energy_Analysis_Correct"
HTheta2Theta1_num = [0, B2, N1**2 * k2]
HTheta2Theta1_denom = [J2, B2, N1**2 * k2]
HTheta2Theta1 = signal.TransferFunction(HTheta2Theta1_num, HTheta2Theta1_denom)

# Torque to buoy angle transfer function (theta1 / tau)
# Equation 4 Jantzen "Energy Analysis Correct"
A = [B2**2, 2 * N1**2 * B2 * k2, N1**4 * k2**2]
B = [J2, B2, N1**2 * k2]
C = [J1, (B1 + B2), +k1 + N1**2 * k2]

# 1 / (C - A/B) == B / (CB - A)
HTorqueTheta1 = signal.TransferFunction(B, np.polysub(np.polymul(C, B), A))

# Torque to flywheel angle transfer function
# theta_2 / tau = (theta_2 / theta_1) * (theta_1 / tau)
HTorqueTheta2 = signal.TransferFunction(
    np.polymul(HTorqueTheta1.num, HTheta2Theta1.num),
    np.polymul(HTorqueTheta1.den, HTheta2Theta1.den),
)

# Torque to generator angle transfer function
# gen / tau = (theta_1 - theta_2) * N2
# Equation on page 3 of Jantzen "Energy Analysis Correct"
deltaAngNum, deltaAngDen = subPolyFrac(
    HTorqueTheta1.num, HTorqueTheta1.den, HTorqueTheta2.num, HTorqueTheta2.den
)
HTorqueGen = signal.TransferFunction(deltaAngNum, deltaAngDen)  # N2 = 1

# find derivative by multiplying transfer function by 's'
# equivalent of shifting the numerator coefficients one place in the array
GenVelNum = np.pad(HTorqueGen.num, (0, 1), "constant")
HTorqueGenVel = signal.TransferFunction(GenVelNum, HTorqueGen.den)
_, comp_gen_vel, _ = HTorqueGenVel.output(comp_exc_torque, comp_t)
_, comp_pto_pitch, _ = HTorqueGen.output(comp_exc_torque, comp_t)
_, comp_buoy_pitch, _ = HTorqueTheta1.output(comp_exc_torque, comp_t)
_, comp_fw_pitch, _ = HTorqueTheta2.output(comp_exc_torque, comp_t)
# remove everything except the last part of the results to remove transient
comp_gen_vel = comp_gen_vel[-len_t:]
comp_pto_pitch = comp_pto_pitch[-len_t:]
comp_buoy_pitch = comp_buoy_pitch[-len_t:]
comp_fw_pitch = comp_fw_pitch[-len_t:]
comp_pto_trq = -comp_gen_vel * flywheel_props['Motor damping']
comp_mech_power = comp_pto_trq * comp_gen_vel
comp_pto_pos = np.rad2deg(comp_pto_pitch) / n_default**2
comp_buoy_pos = np.rad2deg(comp_buoy_pitch)
comp_fw_pos = np.rad2deg(comp_fw_pitch)


In [ ]:
fig, ax = plt.subplots(nrows=6, sharex=True, figsize=(12, 12))
t = wec_tdom.time.values

# Excitation
ax[0].plot(t, comp_pto_trq, label='Reference')
ax[0].plot(t, pto_force, linestyle='dashed', label='WecOptTool')
ax[0].set_ylabel('Torque [N-rad]')
ax[0].set_title('PTO Torque')
ax[0].legend()

# Buoy Position
ax[1].plot(t, comp_buoy_pos, label='Reference')
(wec_tdom.pos*180/np.pi).plot(ax=ax[1], hue='influenced_dof', linestyle='dashed', label='WecOptTool')
ax[1].set_title('Buoy Position')
ax[1].set_ylabel('Position [deg]')
ax[1].legend()

# Flywheel Position
ax[2].plot(t, comp_fw_pos, label='Reference')
ax[2].plot(t, fw_pos*180/np.pi, linestyle='dashed', label='WecOptTool')
ax[2].set_title('Flywheel Position')
ax[2].set_ylabel('Position [deg]')
ax[2].legend()

# PTO Position
ax[3].plot(t, comp_pto_pos, label='Reference')
ax[3].plot(t, pto_pos*180/np.pi, linestyle='dashed', label='WecOptTool')
# ax[3].plot(t, np.ones(len(t))*45)
ax[3].set_title('PTO Position')
ax[3].set_ylabel('Position [deg]')
ax[3].legend()

# PTO Velocity
ax[4].plot(t, comp_gen_vel, label='Reference')
ax[4].plot(t, pto_vel, linestyle='dashed', label='WecOptTool')
ax[4].set_title('PTO Velocity')
ax[4].set_ylabel('Velocity [rad/s]')
ax[4].legend()

# Mechanical Power
ax[5].plot(t, comp_mech_power, label='Reference')
ax[5].plot(t, pto_mech_power, linestyle='dashed', label='WecOptTool')
ax[5].set_title('Power Output')
ax[5].set_ylabel('Power [W]')
ax[5].legend()

for axi in ax:
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)

In [ ]:
plt.plot(wec.omega, pto_force_fd)

In [ ]:
# amplitude comparison
print( 'Reference:\n' +
      f'PTO pitch amplitude: {np.max(np.abs(comp_pto_pos))} deg\n' +
      f'Buoy pitch amplitude: {np.max(np.abs(comp_buoy_pos))} deg\n' +
      f'Flywheel pitch amplitude: {np.max(np.abs(comp_fw_pos))} deg\n' +
      f'Generator velocity: {np.max(np.abs(comp_gen_vel))} rad/s\n' +
      f'Average mech. power: {np.mean(comp_mech_power)} W\n'
)


print( 'WecOptTool:\n' +
      f'PTO pitch amplitude: {(np.max(np.abs(pto_pos)*180/np.pi))} deg\n' +
      f'Buoy pitch amplitude: {(np.abs(wec_tdom.pos)*180/np.pi).max().values} deg\n' +
      f'Flywheel pitch amplitude: {np.max(np.abs(fw_pos)*180/np.pi)} deg\n' +
      f'Generator velocity: {np.max(np.abs(pto_vel))} rad/s\n' +
      f'Average mech. power: {np.mean(pto_mech_power)} W'
)

## Part 2

In [ ]:
import xarray as xr

In [ ]:
def design_obj_fun(x):
    global n
    n += 1

    moi = x.squeeze()
    
    def fw_inertia(wec, x_wec, x_opt, waves, nsubsteps=1):
        pos_fw = wec.vec_to_dofmat(x_opt[nstate_pto:])
        acc_fw = np.dot(wec.derivative2_mat, pos_fw)
        time_matrix = wec.time_mat_nsubsteps(nsubsteps)
        acc_fw = np.dot(time_matrix, acc_fw)
        return moi * acc_fw

    def const_flywheel_resid(wec, x_wec, x_opt, waves):
        return (fw_inertia(wec, x_wec, x_opt, waves)
            + f_spring(wec, x_wec, x_opt, waves)
            #   - f_friction(wec, x_wec, x_opt, waves)
            + f_motor(wec, x_wec, x_opt, waves)
            ).flatten()
    
    constraints = [
    # {'type': 'ineq', 'fun': const_peak_torque_pto},
    # {'type': 'ineq', 'fun': const_rms_velocity_pto},
    # {'type': 'ineq', 'fun': const_max_spring_angle},
    {'type': 'eq', 'fun': const_flywheel_resid},
    ]

    wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=np.array([[buoy_props['MOI']]]),
                       hydrostatic_stiffness=k_buoy,
                       f_add=f_add,
                       constraints=constraints,
                       dof_names=bem_data.influenced_dof.values,
                        )
                        
    print(f'\nRun {n} of {N}: Flywheel MOI: {moi:.2f} kg-m^2')
    results = wec.solve(
        waves,
        obj_fun,
        nstate_opt,
        # x_wec_0=np.ones(wec.nstate_wec)*0.8,
        # x_opt_0=np.ones(nstate_opt),
        scale_x_wec=1e1,
        scale_x_opt=1e-2,
        scale_obj=1e-2,
        optim_options=options
    )
    opt_avg_pwr = results.fun
    print(
        f'* Optimal average electrical power: {opt_avg_pwr:.2f} W' 
    )
    x_wec, x_opt = wec.decompose_state(results.x)
    x_wec_jac , x_opt_jac = wec.decompose_state(results.jac)
    ds = xr.Dataset(data_vars=dict(
        x_wec=('wec_state', x_wec),
        x_opt=('opt_state', x_opt),
        x_wec_jac=('wec_state', x_wec_jac),
        x_opt_jac=('opt_state', x_opt_jac),
        fval=results.fun),
        coords=dict(
        wec_state=range(wec.nstate_wec),
        opt_state=range(nstate_opt))
    )
    wot.write_netcdf(os.path.join('data', f'tutorial_4_results_{x.squeeze()}.nc'), ds)
    return opt_avg_pwr

In [ ]:
constraints

### Solve

In [ ]:
min_moi = 150
max_moi = 200
inc = 5
moi_selections = np.arange(min_moi, max_moi+inc, inc)
global n; n = 0
global N; N = len(moi_selections)
ranges = (slice(
    moi_selections[0], moi_selections[-1]+np.diff(moi_selections)[0], np.diff(moi_selections)[0]
    ),
)
# solve
filename = 'data/tutorial_4_elecpwr_cons_results.nc'
try:
    opt_results = wot.read_netcdf(filename)
except:
    combined_results = []
    opt_x0, opt_fval, x0s, fvals = brute(
        func=design_obj_fun,
        ranges=ranges,
        full_output=True,
        finish=None)
    for x in moi_selections:
        run_filename = os.path.join('data', f'tutorial_4_results_{x}.nc')
        combined_results.append(wot.read_netcdf(run_filename))
        os.remove(run_filename)
    opt_results = xr.concat(combined_results, dim='x0s')
    wot.write_netcdf(filename, opt_results)

In [ ]:
rsnc = k_spring / omega_r**2

In [ ]:
opt_results = wot.read_netcdf('data/tutorial_4_elecpwr_cons_results.nc')

In [ ]:
nsubsteps = 5

max_poss = np.zeros(len(moi_selections))
max_vels = np.zeros(len(moi_selections))
max_frcs = np.zeros(len(moi_selections))
for x in opt_results.x0s:
    xw = opt_results.x_wec.sel(x0s=x).values
    xo = opt_results.x_opt.sel(x0s=x).values
    pto_pos = rel_position(wec, xw, xo, waves, nsubsteps) / n_default**2
    pto_vel = rel_velocity(wec, xw, xo, waves, nsubsteps)
    pto_force = f_motor(wec, xw, xo, waves, nsubsteps)
    max_poss[x] = np.max(np.abs(pto_pos)*180/np.pi)
    max_vels[x] = np.max(np.abs(pto_vel))
    max_frcs[x] = np.max(np.abs(pto_force))
    # pto_force_fd = wec.td_to_fd(pto_force[::nsubsteps])
    # pto_mech_power = mechanical_power(wec, xw, xo, waves, nsubsteps)
    # fw_pos = np.dot(wec.time_mat_nsubsteps(nsubsteps), x_opt[nstate_pto:])
    # plt.plot(t, pto_vel, label=f'MOI = {moi_selections[x]}')
plt.figure()
fig, ax = plt.subplots(2,2)
fig.suptitle('Optimization Results, Mech. Power, No Constraints')
fig.tight_layout()
ax[0,0].plot(moi_selections, max_poss, 'k')
ax[0,0].scatter(moi_selections, max_poss)
ax[0,0].axvline(rsnc, color='r', linestyle='dashed')
ax[0,0].set_title('Max PTO Position')

ax[1,0].plot(moi_selections, max_vels, 'k')
ax[1,0].scatter(moi_selections, max_vels)
ax[1,0].axvline(rsnc, color='r', linestyle='dashed')
ax[1,0].set_title('Max PTO Velocity')

ax[0,1].plot(moi_selections, max_frcs, 'k')
ax[0,1].scatter(moi_selections, max_frcs)
ax[0,1].axvline(rsnc, color='r', linestyle='dashed')
ax[0,1].set_title('Max PTO Force')

ax[1,1].plot(moi_selections[opt_results.x0s], opt_results.fval, 'k', zorder=0)
ax[1,1].scatter(moi_selections[opt_results.x0s], opt_results.fval, zorder=1)
ax[1,1].axvline(rsnc, color='r', linestyle='dashed')
ax[1,1].set_title('Average Power')

In [ ]:
pto_pos

In [ ]:
opt_results.x_wec.sel(x0s=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x0s, fvals, 'k', zorder=0)
ax.scatter(x0s, fvals, zorder=1)

ax.set_xlabel('Flywheel Pitch Moment of Inertia [kg-m^2]')
ax.set_ylabel('Average Power [W]')
ax.set_title('Design optimization results')
fig.tight_layout()